In [61]:
import praw
import re
import json
from datetime import datetime

# Set up Reddit API client
reddit = praw.Reddit(
    client_id="Efbot9iva7bBckLwx-Ic-g",
    client_secret="OWxn-YzLwhj0nDMPb7Ai72gQxVJjmg",
    user_agent="prescribed-burning",  # e.g., "prescribed_burn_study"
)

# List of subreddits you want to search in
subreddits = ["Missoula"]

# List of search terms related to prescribed burns
search_terms = [
    "fire","smoke"
]

# Combine search terms into a single query
search_query = ' OR '.join(search_terms)

# Initialize the results list
results = []

def matches_title(title, search_terms):
    title_lower = title.lower()
    for term in search_terms:
        if term in title_lower:
            # print("term: ",term)
            # print("title: ",title_lower)
            return True
    return False

# Search posts in each subreddit
for subreddit in subreddits:
    print(f"Searching in r/{subreddit}...")
    for post in reddit.subreddit(subreddit).search(
        ' OR '.join(search_terms),  # Combine search terms
        limit=200,                  # Set limit as needed
        sort="new",                 # Sort by newest posts
        time_filter="all"           # Retrieve posts from all time
    ): 
        if matches_title(post.title,search_terms):
            post_data = {
                "title": post.title,
                "score": post.score,
                "date": datetime.utcfromtimestamp(post.created_utc).strftime('%m-%d-%Y'),
                "url": post.url,
                "num_comments": post.num_comments,
                "comments": [],
                "replies": [],
            }
        
            # Fetch comments (replace "more comments" and limit to 0 to get all comments)
            post.comments.replace_more(limit=0)

            # Add comments and replies to the post data
            count = 0
        
            for comment in post.comments.list():  # All comments
                if count >= 1:
                    post_data["replies"].append(comment.body)
                else:
                    post_data["comments"].append(comment.body)
                count += 1

            # Add post data to the results list
            results.append(post_data)

# Save results to a file
import json
with open("reddit_prescribed_burns.json", "w") as f:
    json.dump(results, f, indent=4)

print("Data saved to reddit_prescribed_burns.json")

Searching in r/Missoula...
Data saved to reddit_prescribed_burns.json
